In [1]:
from openai import OpenAI
import chromadb
from chromadb.config import Settings
from dotenv import load_dotenv
import os
import json

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# OpenAI 클라이언트
client = OpenAI(api_key=OPENAI_API_KEY)

# ChromaDB 초기화
chroma_client = chromadb.PersistentClient(
    path="./vector_db",
    settings=Settings(anonymized_telemetry=False)
)

collection = chroma_client.get_or_create_collection(
    name="patent_vectors",
    metadata={"hnsw:space": "cosine"}  # 코사인 유사도 기반
)

In [ ]:
# ===== 세션 상태 =====
session_state = {
    "history": [],  # 최근 대화 기록 (user / assistant 쌍)

    "last_patent_search": {
        "query": None,
        "keywords": [],
        "results": []  # 나중에 벡터 DB 검색 결과 넣을 자리
    },

    "last_ipc_result": {
        "input_text": None,
        "predicted_ipc_codes": [],
        "explanations": []
    }
}

MAX_HISTORY_TURNS = 10  # 최근 10턴까지만 유지  

In [3]:
def add_turn_to_history(state: dict, user_input: str, assistant_output: str) -> None:
    """
    세션 상태에 (user_input, assistant_output) 한 턴을 추가.
    너무 길어지면 최근 MAX_HISTORY_TURNS만 남겨둠.
    """
    state["history"].append({
        "user": user_input,
        "assistant": assistant_output
    })

    if len(state["history"]) > MAX_HISTORY_TURNS:
        state["history"] = state["history"][-MAX_HISTORY_TURNS:]

In [4]:
# ===== intent 분류 =====
def classify_intent_with_llm(user_input: str) -> str:
    prompt = f"""
    너는 특허 질의응답 시스템의 분류기야.
    아래 사용자 입력이 어떤 카테고리에 속하는지 하나만 답해줘.

    카테고리:
    1) PATENT_SEARCH: 비슷한 특허를 검색하거나, 기술 설명을 기반으로 유사 특허를 찾는 요청
    2) IPC_QUERY: IPC 코드(국제특허분류)에 대해 묻거나 특정 발명에 IPC가 어떻게 되는지 묻는 요청
    3) PATENT_FOLLOWUP: 방금 찾은 특허 검색 결과(목록, 번호, 내용 등)에 대해 추가 질문/요청을 하는 경우
    4) OTHER: 그 외 모든 질문 (일반 설명, 잡담, 기타)

    사용자 입력:
    \"\"\" 
    {user_input}
    \"\"\"

    출력 형식:
    PATENT_SEARCH 또는 IPC_QUERY 또는 PATENT_FOLLOWUP 또는 OTHER
    """

    res = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    intent = res.choices[0].message.content.strip()
    return intent

In [5]:
# ===== 특허 검색용 키워드 추출 =====
def extract_core_keywords_from_tech_doc(tech_text: str, max_keywords: int = 5) -> list[str]:
    """
    기술문서(초록 + 청구항 등)에서 특허 검색용 핵심 키워드들을 뽑아서 리스트로 반환.
    """
    prompt = f"""
    너는 특허 검색을 위한 키워드 추출기야.

    아래 기술 설명을 읽고, 이 발명을 다른 일반적인 특허들과
    구분해 줄 수 있는 **핵심 기술 포인트**를 {max_keywords}개 이하의 키워드로 뽑아줘.

    규칙:
    - "전자 장치", "무선 통신 회로", "센서", "메모리"처럼
      거의 모든 전자기기에 공통적으로 들어가는 일반적인 구성 요소 이름만 단독으로 쓰지 마.
    - 대신, 이 발명에서 특히 중요한 동작 흐름/조건/조합을 담아서 써.
      예: "V2V 통신용 자원 패턴 결정", "SPS 요청 포함 보조 정보 전송" 같은 식.
    - 각 키워드는 1줄에 하나씩만 쓰고, 설명 문장 대신 검색에 쓸 수 있는 짧은 구문 형태로 써.

    기술 설명:
    \"\"\" 
    {tech_text}
    \"\"\" 

    출력 형식(중요):
    - 반드시 JSON 배열 형태만 출력해.
    - 절대로 ``` 같은 마크다운 코드블록을 쓰지 마.
    - 'json'이라는 단어도 쓰지 마.
    - 예시: [
        "V2V 통신용 자원 패턴 결정",
        "반지속적 스케줄(SPS) 요청 포함 보조 정보 전송",
        "MAC/RRC 메시지에서 자원 패턴 관련 정보 전송"
      ]
    """

    res = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    raw = res.choices[0].message.content.strip()

    # JSON 형식 파싱 시도
    try:
        keywords = json.loads(raw)
        # 혹시 문자열 하나만 오거나 이상한 형식이면 보정
        if isinstance(keywords, str):
            keywords = [keywords]
        elif not isinstance(keywords, list):
            keywords = [str(keywords)]
    except json.JSONDecodeError:
        # JSON 실패 시 줄 단위 분리로 폴백
        keywords = [
            line.strip("-• ").strip()
            for line in raw.split("\n")
            if line.strip()
        ]

    # 공백 제거 및 빈 값 제거
    cleaned = [k.strip() for k in keywords if str(k).strip()]

    # max_keywords 개수 제한
    return cleaned[:max_keywords]

In [6]:
def decide_max_keywords(text: str) -> int:
    """
    입력 텍스트 길이에 따라 뽑을 키워드 개수 결정.
    """
    length = len(text)

    if length < 300:      # 짧은 설명
        return 3
    elif length < 1500:   # 초록 + 몇 개 청구항
        return 5
    else:                 # 청구항 10개 이상, 긴 문서
        return 7

In [7]:
def extract_keywords_if_patent_search(user_input: str, max_keywords: int | None = None) -> dict:
    """
    1) 사용자의 입력 의도를 LLM으로 분류하고
    2) PATENT_SEARCH일 때만 기술문서에서 키워드를 추출해 반환.

    반환 형식 예시:
    - {"intent": "PATENT_SEARCH", "keywords": ["...", "..."]}
    - {"intent": "IPC_QUERY", "keywords": []}
    - {"intent": "PATENT_FOLLOWUP", "message": "..."}
    - {"intent": "OTHER", "message": "..."}
    """
    intent = classify_intent_with_llm(user_input)

    if intent == "PATENT_SEARCH":
        if max_keywords is None:
            max_keywords = decide_max_keywords(user_input)

        keywords = extract_core_keywords_from_tech_doc(
            user_input,
            max_keywords=max_keywords
        )

        # 👉 여기서 state에 "마지막 특허 검색 정보" 저장
        session_state["last_patent_search"]["query"] = user_input
        session_state["last_patent_search"]["keywords"] = keywords
        # session_state["last_patent_search"]["results"] 는
        # 실제 벡터 DB 검색 붙일 때 여기에 채우면 됨.

        return {"intent": intent, "keywords": keywords}

    elif intent == "IPC_QUERY":
        # IPC 관련 처리는 나중에 구현 (여기서 state 저장도 가능)
        return {"intent": "IPC_QUERY", "message": "IPC 기능은 아직 미구현입니다."}

    elif intent == "PATENT_FOLLOWUP":
        return {"intent": "PATENT_FOLLOWUP", "message": "방금 검색한 특허 결과를 기반으로 후속 질의를 처리해야 합니다."}

    else:
        return {"intent": "OTHER", "message": "해당 질문은 범위 밖입니다."}

In [8]:
# ===== (예시) 사용자 입력 한 턴 처리하는 상위 함수 =====
def handle_user_input(user_input: str) -> str:
    """
    한 턴의 사용자 입력을 받아서:
    1) intent 분류
    2) intent에 따라 처리
    3) state 업데이트 (history 포함)
    4) 최종 assistant 답변 텍스트 반환
    """
    intent = classify_intent_with_llm(user_input)

    # 일단 결과 문장은 간단하게만 구성 (나중에 RAG, IPC, followup 붙이면 확장)
    if intent == "PATENT_SEARCH":
        max_k = decide_max_keywords(user_input)
        keywords = extract_core_keywords_from_tech_doc(user_input, max_keywords=max_k)

        # state 업데이트
        session_state["last_patent_search"]["query"] = user_input
        session_state["last_patent_search"]["keywords"] = keywords
        # TODO: 여기서 벡터 DB 검색을 하고 results 채우면 됨.
        session_state["last_patent_search"]["results"] = []  # 임시

        assistant_reply = (
            "요청하신 기술 설명은 특허 검색용으로 다음과 같은 핵심 키워드로 요약할 수 있어요:\n"
            + "\n".join(f"- {k}" for k in keywords)
            + "\n\n(벡터 DB 검색은 아직 연결 전입니다.)"
        )

    elif intent == "IPC_QUERY":
        # TODO: IPC 추천/설명 로직 구현 후 state에 last_ipc_result 저장
        assistant_reply = "IPC 관련 질의로 분류되었지만, IPC 처리 로직은 아직 구현되지 않았어요."

    elif intent == "PATENT_FOLLOWUP":
        # TODO: session_state["last_patent_search"] 를 사용해서 후속 Q&A 처리
        assistant_reply = (
            "방금 검색한 특허 결과들을 기반으로 한 후속 질문으로 보이는데, "
            "follow-up 처리 로직은 아직 구현 중이에요."
        )

    else:  # OTHER
        # 그냥 일반 LLM 챗봇처럼 답변
        res = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": user_input}]
        )
        assistant_reply = res.choices[0].message.content.strip()

    # 마지막으로 history에 (입력/출력) 한 턴 저장
    add_turn_to_history(session_state, user_input, assistant_reply)

    return assistant_reply

In [9]:
if __name__ == "__main__":
    # 간단 테스트
    user_input = """
    본 발명은 인공지능 컴퓨터 비전을 기반으로, 대상자의 동공 움직임과 마이크로사케이드(microsaccade)를 정밀하게 분석하여 알츠하이머를 조기 진단하는 시스템에 관한 것입니다.
    이는 대상자의 미세 안구 움직임 패턴을 정량적으로 측정하고, 이를 AI가 정상인 데이터와 비교하여 초기 알츠하이머 징후를 판단하는 것을 그 특징으로 합니다.
    따라서 본 발명은 기존 검사 방식 대비 비용이 저렴하고, 검사 시간이 짧으며, 비침습적으로 진단이 가능하여 사용 편의성과 효율성을 크게 향상시키는 효과가 있습니다.
    
    이 특허를 출원하려고 하는데 비슷한 특허가 있는지 찾아봐줘.
    """

    reply = handle_user_input(user_input)
    print("=== Assistant ===")
    print(reply)
    print("\n=== Session State ===")
    print(json.dumps(session_state, ensure_ascii=False, indent=2))


=== Assistant ===
요청하신 기술 설명은 특허 검색용으로 다음과 같은 핵심 키워드로 요약할 수 있어요:
- 동공 움직임 및 마이크로사케이드 분석
- AI 기반 알츠하이머 조기 진단
- 미세 안구 움직임 패턴 정량적 측정
- 비침습적 진단 시스템
- 비용 효율적 검사 방식

(벡터 DB 검색은 아직 연결 전입니다.)

=== Session State ===
{
  "history": [
    {
      "user": "\n    본 발명은 인공지능 컴퓨터 비전을 기반으로, 대상자의 동공 움직임과 마이크로사케이드(microsaccade)를 정밀하게 분석하여 알츠하이머를 조기 진단하는 시스템에 관한 것입니다.\n    이는 대상자의 미세 안구 움직임 패턴을 정량적으로 측정하고, 이를 AI가 정상인 데이터와 비교하여 초기 알츠하이머 징후를 판단하는 것을 그 특징으로 합니다.\n    따라서 본 발명은 기존 검사 방식 대비 비용이 저렴하고, 검사 시간이 짧으며, 비침습적으로 진단이 가능하여 사용 편의성과 효율성을 크게 향상시키는 효과가 있습니다.\n\n    이 특허를 출원하려고 하는데 비슷한 특허가 있는지 찾아봐줘.\n    ",
      "assistant": "요청하신 기술 설명은 특허 검색용으로 다음과 같은 핵심 키워드로 요약할 수 있어요:\n- 동공 움직임 및 마이크로사케이드 분석\n- AI 기반 알츠하이머 조기 진단\n- 미세 안구 움직임 패턴 정량적 측정\n- 비침습적 진단 시스템\n- 비용 효율적 검사 방식\n\n(벡터 DB 검색은 아직 연결 전입니다.)"
    }
  ],
  "last_patent_search": {
    "query": "\n    본 발명은 인공지능 컴퓨터 비전을 기반으로, 대상자의 동공 움직임과 마이크로사케이드(microsaccade)를 정밀하게 분석하여 알츠하이머를 조기 진단하는 시스템에 관한 것입니다.\n    이는 대상자의 미세 안구 움직임 패턴을 정량적으로 측정하고, 이를 AI가 정상인 데이터와 비교

In [11]:
second_input = "특허 출원 전반적인 과정에 관해서 설명해줄래?."

second_reply = handle_user_input(second_input)

print("=== Assistant (2번째 턴) ===")
print(second_reply)

=== Assistant (2번째 턴) ===
물론입니다. 특허 출원 과정은 일반적으로 다음과 같은 단계를 포함합니다. 각 국가마다 세부사항은 다를 수 있지만, 기본적인 절차는 유사합니다.

1. **아이디어 개발 및 조사**:
   - 먼저, 특허를 받고자 하는 아이디어나 발명을 명확히 정의해야 합니다.
   - 신규성, 비공식성, 실용성을 확인하기 위해 기존 특허 및 문헌 조사를 실시합니다. 이를 특허조사 또는 선행기술 조사라고 합니다.

2. **특허 명세서 작성**:
   - 특허 명세서는 발명의 상세한 설명이 포함된 문서로, 발명의 구성 요소, 작동 방식, 구현 방법 등을 명확히 기술해야 합니다.
   - 발명의 권리 범위를 명시하는 청구항(claim)을 작성하는 것이 중요합니다.

3. **특허 출원 제출**:
   - 작성된 명세서를 특허청에 제출합니다. 이때 출원서 및 일정한 수수료도 함께 제출해야 합니다.
   - 출원 유형에 따라, 임시(프로비셔널) 출원과 정식(비프로비셔널) 출원으로 나눌 수 있습니다.

4. **출원 공개**:
   - 특허 출원은 대부분 출원일로부터 18개월 이후에 공개됩니다. 이 단계에서 출원한 내용이 일반에 공개되어, 공개발명의 법적 보호가 시작됩니다.

5. **심사 요청 및 심사**:
   - 특허청의 심사관에 의한 심사가 시작됩니다. 심사 과정에서는 발명의 신규성, 진보성, 산업적 이용 가능성 등을 평가합니다.
   - 심사 과정 중에 거절 이유가 있을 경우, 이를 극복하기 위해 출원인은 의견서를 제출하거나 청구항을 수정할 수 있습니다.

6. **등록 결정 및 등록료 납부**:
   - 심사 결과 모든 조건이 충족되면, 특허로 등록할 수 있는 최종 결정이 내려집니다.
   - 등록료를 납부하면 특허가 정식으로 등록되고 발명에 대한 독점권이 부여됩니다.

7. **유지 및 갱신**:
   - 등록된 특허권은 원칙적으로 출원일로부터 20년간 보호됩니다. 이를 유지하기 위해 주기적으로 갱신료를 납부해야 합니다.

특

In [12]:
import json

print("=== Session History ===")
print(json.dumps(session_state["history"], ensure_ascii=False, indent=2))

=== Session History ===
[
  {
    "user": "\n    본 발명은 인공지능 컴퓨터 비전을 기반으로, 대상자의 동공 움직임과 마이크로사케이드(microsaccade)를 정밀하게 분석하여 알츠하이머를 조기 진단하는 시스템에 관한 것입니다.\n    이는 대상자의 미세 안구 움직임 패턴을 정량적으로 측정하고, 이를 AI가 정상인 데이터와 비교하여 초기 알츠하이머 징후를 판단하는 것을 그 특징으로 합니다.\n    따라서 본 발명은 기존 검사 방식 대비 비용이 저렴하고, 검사 시간이 짧으며, 비침습적으로 진단이 가능하여 사용 편의성과 효율성을 크게 향상시키는 효과가 있습니다.\n\n    이 특허를 출원하려고 하는데 비슷한 특허가 있는지 찾아봐줘.\n    ",
    "assistant": "요청하신 기술 설명은 특허 검색용으로 다음과 같은 핵심 키워드로 요약할 수 있어요:\n- 동공 움직임 및 마이크로사케이드 분석\n- AI 기반 알츠하이머 조기 진단\n- 미세 안구 움직임 패턴 정량적 측정\n- 비침습적 진단 시스템\n- 비용 효율적 검사 방식\n\n(벡터 DB 검색은 아직 연결 전입니다.)"
  },
  {
    "user": "방금 기술 설명이 실제로는 어떤 산업 분야에서 많이 활용될 것 같은지도 설명해줘.",
    "assistant": "방금 검색한 특허 결과들을 기반으로 한 후속 질문으로 보이는데, follow-up 처리 로직은 아직 구현 중이에요."
  },
  {
    "user": "특허 출원 전반적인 과정에 관해서 설명해줄래?.",
    "assistant": "물론입니다. 특허 출원 과정은 일반적으로 다음과 같은 단계를 포함합니다. 각 국가마다 세부사항은 다를 수 있지만, 기본적인 절차는 유사합니다.\n\n1. **아이디어 개발 및 조사**:\n   - 먼저, 특허를 받고자 하는 아이디어나 발명을 명확히 정의해야 합니다.\n   - 신규성, 비공식성, 실용성을 확인하기 위해 기존 특허 및 문헌 조사를 실시

In [13]:
print("=== Last Patent Search ===")
print(json.dumps(session_state["last_patent_search"], ensure_ascii=False, indent=2))

=== Last Patent Search ===
{
  "query": "\n    본 발명은 인공지능 컴퓨터 비전을 기반으로, 대상자의 동공 움직임과 마이크로사케이드(microsaccade)를 정밀하게 분석하여 알츠하이머를 조기 진단하는 시스템에 관한 것입니다.\n    이는 대상자의 미세 안구 움직임 패턴을 정량적으로 측정하고, 이를 AI가 정상인 데이터와 비교하여 초기 알츠하이머 징후를 판단하는 것을 그 특징으로 합니다.\n    따라서 본 발명은 기존 검사 방식 대비 비용이 저렴하고, 검사 시간이 짧으며, 비침습적으로 진단이 가능하여 사용 편의성과 효율성을 크게 향상시키는 효과가 있습니다.\n\n    이 특허를 출원하려고 하는데 비슷한 특허가 있는지 찾아봐줘.\n    ",
  "keywords": [
    "동공 움직임 및 마이크로사케이드 분석",
    "AI 기반 알츠하이머 조기 진단",
    "미세 안구 움직임 패턴 정량적 측정",
    "비침습적 진단 시스템",
    "비용 효율적 검사 방식"
  ],
  "results": []
}


In [14]:
followup_input = "방금 뽑아준 키워드들 중에서 'AI 기반 알츠하이머 조기 진단'이랑 가장 직접적으로 관련된 부분만 다시 정리해줘."

followup_reply = handle_user_input(followup_input)

print("=== Assistant (follow-up 턴) ===")
print(followup_reply)

print("=== Session History (다시 확인) ===")
print(json.dumps(session_state["history"], ensure_ascii=False, indent=2))

=== Assistant (follow-up 턴) ===
방금 검색한 특허 결과들을 기반으로 한 후속 질문으로 보이는데, follow-up 처리 로직은 아직 구현 중이에요.
=== Session History (다시 확인) ===
[
  {
    "user": "\n    본 발명은 인공지능 컴퓨터 비전을 기반으로, 대상자의 동공 움직임과 마이크로사케이드(microsaccade)를 정밀하게 분석하여 알츠하이머를 조기 진단하는 시스템에 관한 것입니다.\n    이는 대상자의 미세 안구 움직임 패턴을 정량적으로 측정하고, 이를 AI가 정상인 데이터와 비교하여 초기 알츠하이머 징후를 판단하는 것을 그 특징으로 합니다.\n    따라서 본 발명은 기존 검사 방식 대비 비용이 저렴하고, 검사 시간이 짧으며, 비침습적으로 진단이 가능하여 사용 편의성과 효율성을 크게 향상시키는 효과가 있습니다.\n\n    이 특허를 출원하려고 하는데 비슷한 특허가 있는지 찾아봐줘.\n    ",
    "assistant": "요청하신 기술 설명은 특허 검색용으로 다음과 같은 핵심 키워드로 요약할 수 있어요:\n- 동공 움직임 및 마이크로사케이드 분석\n- AI 기반 알츠하이머 조기 진단\n- 미세 안구 움직임 패턴 정량적 측정\n- 비침습적 진단 시스템\n- 비용 효율적 검사 방식\n\n(벡터 DB 검색은 아직 연결 전입니다.)"
  },
  {
    "user": "방금 기술 설명이 실제로는 어떤 산업 분야에서 많이 활용될 것 같은지도 설명해줘.",
    "assistant": "방금 검색한 특허 결과들을 기반으로 한 후속 질문으로 보이는데, follow-up 처리 로직은 아직 구현 중이에요."
  },
  {
    "user": "특허 출원 전반적인 과정에 관해서 설명해줄래?.",
    "assistant": "물론입니다. 특허 출원 과정은 일반적으로 다음과 같은 단계를 포함합니다. 각 국가마다 세부사항은 다를 수 있지만, 기본적인 절차는 유사합니다.\n\n1. **아이디